In [1]:
USE_SAME_FEW_SHOTS=True

In [2]:
number_of_shots = 7

In [3]:
# model_name = "meta-llama/Llama-2-7b-chat-hf"
model_name = "meta-llama/Llama-2-13b-chat-hf"

In [4]:
!pip install -q openai --progress-bar off

In [5]:
!pip install -q langchain --progress-bar off

In [6]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

# import openai
# from openai import OpenAI

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [7]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [8]:
output_datapath = "data/output/2023.11.28-6_Llama2_13B_API_7-few-shot_SEMANTIC_SIMILARITY/"

In [9]:
# # llm = openai.ChatCompletion.create(
# #     model = 'meta-llama/Llama-2-7b-chat-hf', 
# #     messages = [{"role": "system", "content": prompt}],
# #     max_tokens = 150,
# #     response_format = {"type": "json_object"},
# #     stream = True
# # )

# client = OpenAI()

# chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": "Say this is a test",
#         }
#     ],
#     model='meta-llama/Llama-2-7b-chat-hf',
# )

In [10]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [11]:
# if USE_SAME_FEW_SHOTS==True:
#     data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0'])
#     data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
#         (data_sel_4_few_shot_examples.text_id==5614) |
#         (data_sel_4_few_shot_examples.text_id==2089) |
#         (data_sel_4_few_shot_examples.text_id==5665) |
#         (data_sel_4_few_shot_examples.text_id==370)  |
#         (data_sel_4_few_shot_examples.text_id==1422) |
#         (data_sel_4_few_shot_examples.text_id==4329) |
#         (data_sel_4_few_shot_examples.text_id==4929)
#     ]
# else:
#     data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0']).sample(5)

# data_sel_4_few_shot_examples

In [12]:
top7_train_data_sim_to_each_test_data_filename = "../data/input/top7_train_data_sim_to_each_test_data.pickle"

with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [13]:
# few_shots = ""

# for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
#     input_text = "Input text: \""+row['text']+"\"\n"
#     output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    
#     few_shots += input_text
#     few_shots += output_text
    
# print(few_shots)

In [14]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx")
data = data.drop(columns=['Unnamed: 0']) #, 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
data

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0


In [15]:
data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx")

In [16]:
# template_part1 = """[INST]
# <<SYS>>
# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG 0---".

# The following are 7 examples of expected classification given an input text:

# """+few_shots+"""

# The text to be processed is:
# <</SYS>>

# '''
# """

# template_part2 = """
# '''

# <<SYS>>
# Answer with label only usign the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

In [17]:
#data.iloc[181:183]

In [18]:
# data_sel = data #.sample(2)
data_sel = data #.iloc[54:] #.sample(2)

In [19]:
# body = {
#   "model": "meta-llama/Llama-2-7b-chat-hf",
#   "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Say 'test'."}],
#   "temperature": 0.0
# }

# with s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body) as resp:
#     print(resp.json())

In [20]:
%%time
#results = data_sel.apply(lambda x: llm(prompt.format(text=x["text"])), axis=1)

cnt = 0
# cnt = 54

results = []

for index, row in data_sel.iterrows():

  #print("\n\n##################################################")

  #print("EXPECTED_SDG:", row["sdg"], "TEXT:", row["text"].replace('\n',''),'\n\n')

  few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:number_of_shots]

  few_shots = ""

  for train_data_id in few_shots_list:

    # print("##################################################")
    # print(train_data_id)

    train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]

    # print(train_row)

    # print(train_row.text.values[0])
    # print(train_row.sdg.values[0])

    input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
    output_text = "Output: \"---SDG "+str(train_row.sdg.values[0])+'---\"\n\n'

    few_shots += input_text
    few_shots += output_text

  #print(few_shots)

  template = """[INST]
<<SYS>>
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The following are """+str(number_of_shots)+""" non exhaustive examples of expected classification given an input text:

"""+few_shots+"""


The text to be processed is:
<</SYS>>

'''
"""+row["text"]+"""
'''

<<SYS>>
Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""



  # print(template)

  text_id = row['text_id']
  text = row["text"]
  sdg_true = row["sdg"]

  #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
  query = template

  body = {
    "model": model_name,
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."}, 
        {"role": "user", "content": query}],
    "temperature": 0.0
  }


  try:
      result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
      print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'].replace("\n"," - "))
  except Exception as e:
#       print("#############################################################################")
#       print(e, result)
      print(cnt, text_id, sdg_true, "ERROR:"+str(e)+str(result))
#       print("#############################################################################")
  #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    
  result_extended = (cnt, text_id, sdg_true, result)

  #print(result_extended)

  with open(output_datapath+f'/sdg_classification_Llama2_7B_SEMANTIC_SIMILARITY_{cnt}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)

  results.append(result_extended)

  cnt+=1

  # if cnt==5:
  #   break

0 6668 0   Sure, I'd be happy to help! Based on the text you provided, I would classify it as follows: -  - ---SDG 0--- -  - The text does not mention any specific sustainable development goals, and the content is focused on a subway derailment in Brooklyn with no reported injuries. Therefore, it does not align with any of the SDGs.
1 6099 0   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 11--- -  - The text is classified under SDG 11, as it mentions "tourism" and "leisure" activities, which are related to sustainable cities and communities.
2 6471 0   Sure! Here's the label for the given text: -  - ---SDG 16---
3 1786 0   Sure! I'd be happy to help. -  - The text you provided is: -  - "Four minor planets named after Indian scientists, one of them a Malayali -  - The Malayali in the group is Aswin Sekhar, a 38-year-old Palakkad native who has been credited for the discovery of asteroid 2000LJ27. -  - According to the WGSBN bulletin, Aswin is the fi

21 3889 0 ERROR:'choices'{'generated_text': None, 'embedding_outputs': None, 'num_input_tokens': None, 'num_input_tokens_batch': None, 'num_generated_tokens': None, 'num_generated_tokens_batch': None, 'preprocessing_time': None, 'generation_time': None, 'timestamp': 1701194770.740796, 'finish_reason': None, 'error': {'message': 'rayllm.backend.llm.error_handling.PromptTooLongError: Input too long. Recieved 4826 tokens, but the maximum input length is 4096 tokens. (Request ID: 76c617dcfc564a2d535713e4a357268c)', 'internal_message': 'rayllm.backend.server.openai_compat.openai_exception.OpenAIHTTPException (Request ID: 76c617dcfc564a2d535713e4a357268c)', 'code': 400, 'type': 'OpenAIHTTPException', 'param': {}}, 'num_total_tokens': 0, 'num_total_tokens_batch': 0, 'total_time': None}
22 6576 0   Sure, I'd be happy to help! Based on the text you provided, I would classify it as follows: -  - ---SDG 3--- -  - The text mentions "medical innovations" and "health care spending," which aligns wit

37 4226 0 ERROR:'choices'{'generated_text': None, 'embedding_outputs': None, 'num_input_tokens': None, 'num_input_tokens_batch': None, 'num_generated_tokens': None, 'num_generated_tokens_batch': None, 'preprocessing_time': None, 'generation_time': None, 'timestamp': 1701194805.4570937, 'finish_reason': None, 'error': {'message': 'rayllm.backend.llm.error_handling.PromptTooLongError: Input too long. Recieved 4287 tokens, but the maximum input length is 4096 tokens. (Request ID: 26af0e175294f339b4f8d7c631d256c0)', 'internal_message': 'rayllm.backend.server.openai_compat.openai_exception.OpenAIHTTPException (Request ID: 26af0e175294f339b4f8d7c631d256c0)', 'code': 400, 'type': 'OpenAIHTTPException', 'param': {}}, 'num_total_tokens': 0, 'num_total_tokens_batch': 0, 'total_time': None}
38 2335 0   Sure! I'd be happy to help. -  - The text you provided is: -  - "Dodgers To Place Chris Taylor On IL  MLB Trade RumorsDodgers place outfielder Chris Taylor (knee) on injured list - ESPN  ESPNDodger

51 6224 0   Sure! Here's the text you provided, with the label for each paragraph: -  - ---SDG 0--- -  - Pregnant Kourtney Kardashian, Travis Barker reveal their baby's sex  USA TODAYKourtney Kardashian and Travis Barker host a gender reveal after sharing baby news  Yahoo EntertainmentKourtney Kardashian & Travis Barker Reveal Sex of Their Baby | E! News  E! NewsAll the BTS Pics from Kravis's "Baby Barker World Tour" Reveal Party  CosmopolitanKourtney Kardashian Travis Barker Having a Boy, Gender Reveal Shows  TMZView Full Coverage on Google News -  - ---SDG 0---
52 6243 0   Sure, I'd be happy to help! Here's the text you provided, with my label for each paragraph: -  - ---SDG 16--- -  - New Delhi: Union Minister Bhupender Yadav on Tuesday lashed out at the Opposition for ‘engaging in politics’ over the Uniform Civil Code (UCC), saying that Article 44 of the Constitution has established directive principles for states regarding the UCC. -  - ---SDG 16--- -  - He emphasised that the BJP

65 4176 1   Sure, I'd be happy to help! Here's the text you provided, with my classification of it according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - "As an illustration, nearly 50% of those in the second-poorest strata are in the two richest income quintiles, up from 31% in 2003. Anecdotal evidence suggests that measurement has been altered to widen access to subsidies for political purposes. This calls for a more systematic cross-checking of information, e.g. by using a unique identification number such as the cedula for all social and tax purposes. The Sisben is updated every 3 years, which may not be often enough to reflect changes in the personal situation (e.g. loss of job)." -  - ---SDG 10---
66 6655 1   Sure, I'd be happy to help! Here's the text you provided, with the relevant information highlighted: -  - "Thus, pk (the identification method) identifies person i as poor when the number of weighted deprivations is at

...

901 6606 15   Sure, I'd be happy to help! Here's the label for the given text: -  - ---SDG 13---
902 4075 15   Sure, I'd be happy to help! Here's the text you provided, with the relevant SDG label based on its content: -  - "For example, the importance of biodiversity or ecosystems is recognised in several of the NDPs reviewed, though in some cases this is restricted to general strategic directions. A fewer number of NDPs incorporate specific biodiversity-relevant targets with associated indicators to monitor progress. Examples of biodiversity-relevant targets and indicators that are incorporated in NDPs include rates of deforestation, land use and degradation (Colombia), increase in forest cover (Nepal, Uganda), species in danger of extinction, and the number and size of protected areas." -  - Based on the content of the text, the most appropriate SDG label is: -  - ---SDG 15---
903 3920 15   Sure, I'd be happy to help! Based on the text you provided, the most appropriate SDG label wo

926 1515 15   Sure, I'd be happy to help! Here's the text you provided, with my classification of it according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 15--- -  - The text discusses the fluctuation of roundwood harvesting in Finland due to changes in the international forest-product market situation, and the release of CO2 from organic soils due to drainage of peatland. It also projects that forest soils will become a sink and the forest biomass will remain a sink with increasing rates of removal for 2013-2020. Based on this information, I have classified the text as related to SDG 15, which is focused on life on land.
927 6479 15   Sure, I'd be happy to help! Here's the label for the given text: -  - ---SDG 15---
928 2296 15   Sure, I'd be happy to help! Based on the text you provided, the most appropriate SDG label would be: -  - ---SDG 15--- -  - This is because the text discusses sustainable forest management, the 

949 3474 15   Sure, I'd be happy to help! Based on the text you provided, I would classify it as follows: -  - ---SDG 15--- -  - The text discusses the importance of biodiversity mainstreaming and the need for core indicators to measure progress towards this goal. It highlights the challenges of measuring biodiversity and the need for consistent data collection across countries. Overall, the text is focused on the theme of biodiversity conservation and sustainable use, which is central to SDG 15.
950 518 15   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 15---
951 2066 15   Sure, I'd be happy to help! Here's the label for the given text: -  - ---SDG 15---
952 389 15   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 15---
953 943 15   Sure, I'd be happy to help! Based on the text you provided, the best label would be: -  - ---SDG 15--- -  - This label represents the Sustainable Development Goal 15, which is focuse

977 6715 16   Sure, I'd be happy to help! Based on the text you provided, I would classify it as follows: -  - ---SDG 16--- -  - The text discusses the impact of geopolitics on the response to the COVID-19 pandemic, and how this has influenced international law and global health cooperation. The focus on power dynamics and competition between great powers is consistent with SDG 16, which aims to promote peaceful and inclusive societies.
978 2543 16 ERROR:Expecting value: line 1 column 1 (char 0){'id': 'meta-llama/Llama-2-13b-chat-hf-d95507b3c343a6de70f6d524515a17fe', 'object': 'text_completion', 'created': 1701197111, 'model': 'meta-llama/Llama-2-13b-chat-hf', 'choices': [{'message': {'role': 'assistant', 'content': "  Sure, I'd be happy to help! Based on the text you provided, I would classify it as follows:\n\n---SDG 16---\n\nThe text discusses the impact of geopolitics on the response to the COVID-19 pandemic, and how this has influenced international law and global health cooperati

995 567 16   Sure, I'd be happy to help! Based on the text you provided, I would classify it as follows: -  - ---SDG 16--- -  - The text discusses the interventions of governments on the internet, including mass surveillance, censorship, and the shutdown of the internet in times of turmoil. These interventions are seen as a threat to human rights, particularly freedom of expression and freedom of opinion. The text also highlights the abuse of jurisdictional power and the attacks on these rights. -  - Therefore, the most appropriate SDG label for this text is SDG 16, which focuses on peaceful and inclusive societies, access to justice for all, and effective and accountable institutions.
996 4214 16   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 16---
997 322 16   Sure, I'd be happy to help! Based on the text you provided, I would classify it as follows: -  - ---SDG 16--- -  - The text discusses the relationship between human rights, work, and subje

In [21]:
# import time

# cnt = 0
# # cnt = 181

# results = []
# for index, row in data_sel.iterrows():
    
#     text_id = row['text_id']
#     text = row["text"]
#     sdg_true = row["sdg"]
    
#     query = template_part1 + text + template_part2
    
#     body = {
#       "model": "meta-llama/Llama-2-13b-chat-hf",
#       "messages": [
#           {"role": "system", "content": "You are a helpful assistant."}, 
#           {"role": "user", "content": query}],
#       "temperature": 0.0
#     }
    
        
#     try:
#         result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
#         print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
#     except Exception as e:
#         print("#############################################################################")
#         print(e)
#         print("#############################################################################")
        
    
#     result_extended = (cnt, text_id, sdg_true, result)
    
#     with open(output_datapath+f'sdg_classification_Llama2_13B_{cnt}.pkl', 'wb') as f:
#         pickle.dump(result_extended, f)
    
#     results.append(result_extended)
#     cnt+=1
#     time.sleep(0.5)

In [22]:
files = glob.glob(output_datapath + "*.pkl")
len(files)

1020

In [23]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    try:
        if len(res[3]['choices'])>1:
            raise Exception('ERROR: Multilabel detected!')
            break
        sdg_pred = res[3]['choices'][0]['message']['content']
        res4df = (res[0], res[1], res[2], sdg_pred)
        print(res4df)
        tuple_list.append(res4df)
    except: 
        print("#################################################################")
        print(res)
        print("#################################################################")
        

(289, 1960, 4, "  Sure, I'd be happy to help! Based on the text you provided, I would classify it as follows:\n\n---SDG 4---\n\nThe text discusses the issue of quality in education, specifically the decrease in the percentage of qualified teachers, despite an increase in access to education. This is related to SDG 4, which aims to ensure inclusive and equitable quality education and promote lifelong learning for all.")
(783, 2676, 13, "  Sure, I'd be happy to help! Here's the label for the given text:\n\n---SDG 13---")
(246, 3956, 4, "  Sure, I'd be happy to help! Based on the text you provided, I would classify it as follows:\n\n---SDG 4---\n\nThe text discusses the challenges faced by migrant students in terms of education, including performance gaps and dropout rates. It also mentions the importance of addressing these issues to ensure that all students, regardless of their background, have access to quality education. This aligns with SDG 4, which aims to ensure inclusive and equit

In [24]:
len(tuple_list)

1006

In [25]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [26]:
results_processed

,cnt,id,true_label,predicted_label
0,289,1960,4,"Sure, I'd be happy to help! Based on the tex..."
1,783,2676,13,"Sure, I'd be happy to help! Here's the label..."
2,246,3956,4,"Sure, I'd be happy to help! Based on the tex..."
3,50,2541,0,"Sure, I'd be happy to help! Here's the text ..."
4,288,4205,4,"Sure, I'd be happy to help! Here's the label..."
...,...,...,...,...
1001,691,5312,11,"Sure, I'd be happy to help! Here's the label..."
1002,137,2747,2,"Sure, I'd be happy to help! Based on the tex..."
1003,513,4489,8,"Sure, I'd be happy to help! Here's the text ..."
1004,723,2110,12,"Sure, I'd be happy to help! Here's the label..."


In [27]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Sure, I'd be happy to help! Based on the tex..."
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,"Sure, I'd be happy to help! Here's the label..."
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,Sure! Here's the label for the given text:\n...
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,Sure! I'd be happy to help.\n\nThe text you ...
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,Sure! Here's the label for the given text:\n...
...,...,...,...,...,...,...,...,...,...,...,...
1001,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,"Sure, I'd be happy to help! Here's the label..."
1002,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,"Sure, I'd be happy to help! Based on the tex..."
1003,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,"Sure, I'd be happy to help! Based on the tex..."
1004,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,"Sure, I'd be happy to help! Here's the label..."


In [28]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label,predicted_label_clean
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Sure, I'd be happy to help! Based on the tex...",0
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,"Sure, I'd be happy to help! Here's the label...",11
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,Sure! Here's the label for the given text:\n...,16
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,Sure! I'd be happy to help.\n\nThe text you ...,13
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,Sure! Here's the label for the given text:\n...,7
...,...,...,...,...,...,...,...,...,...,...,...,...
1001,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,"Sure, I'd be happy to help! Here's the label...",16
1002,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,"Sure, I'd be happy to help! Based on the tex...",16
1003,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,"Sure, I'd be happy to help! Based on the tex...",16
1004,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,"Sure, I'd be happy to help! Here's the label...",16


In [29]:
results_processed_merged.groupby("predicted_label_clean").count()

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,
0,22,1,22,22,1,1,1,22,22,22,22
1,73,72,73,73,72,72,72,73,73,73,73
10,53,53,53,53,53,53,53,53,53,53,53
11,64,60,64,64,60,60,60,64,64,64,64
12,69,69,69,69,69,69,69,69,69,69,69
13,67,66,67,67,66,66,66,67,67,67,67
14,57,57,57,57,57,57,57,57,57,57,57
15,62,60,62,62,60,60,60,62,62,62,62
16,59,56,59,59,56,56,56,59,59,59,59


In [30]:
results_processed_merged.to_excel(output_datapath+'report_sdg_classification_Llama2_7B.xlsx')

In [31]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1006
1006
              precision    recall  f1-score   support

           0       0.95      0.46      0.62        46
           1       0.73      0.88      0.80        60
           2       0.89      0.83      0.86        60
           3       0.82      0.93      0.87        60
           4       0.93      0.90      0.92        60
           5       0.92      0.98      0.95        60
           6       1.00      0.97      0.98        60
           7       0.95      0.87      0.90        60
           8       0.75      0.72      0.74        60
           9       0.79      0.83      0.81        60
          10       0.77      0.68      0.73        60
          11       0.86      0.92      0.89        60
          12       0.86      0.98      0.91        60
          13       0.84      0.93      0.88        60
          14       0.98      0.93      0.96        60
          15       0.89      0.92      0.90        60
          16       0.92      0.90      0.91        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
